In [1]:
import sqlalchemy
import db_module

create_new_test_db = True
import os
if os.path.exists("tests.sqlite"): #Если файл уже есть, заморачиваться с перезаполнением не будем
    create_new_test_db = False

## Создаем тестовую БД методом класса dbModule

In [2]:
myDB = db_module.dbModule(dbstring = 'sqlite:///tests.sqlite')
if create_new_test_db:
    myDB.create_sqlite_file()

## Напечатаем названия таблиц в БД для проверки

In [3]:
db_module.Base.metadata.tables.keys()

dict_keys(['image', 'dataset', 'category', 'license', 'annotation', 'trainResult', 'categoryToModel', 'model'])

## Заполним базу аннотациями из COCO -- это необходимо делать один раз

In [4]:
dataDir=r"D:\Work\AutoML\annotations_trainval2017"
dataType='train2017'
annFile=r"{}\annotations\instances_{}.json".format(dataDir,dataType)


import time

begin = time.time()
if create_new_test_db:
    myDB.fill_cats_dogs() #Загружаем базу Kaggle - кошки и собаки
    myDB.fill_coco(annFile, firstTime = True) #Надо указать True чтобы подгрузить категории 
end = time.time()

print('DB filled in', end-begin, 'seconds')

DB filled in 0.0 seconds


## Добавляем информацию о моделях в БД - эта информация может в дальнейшем использоваться для рекомендаций при подготовке новых моделей

In [5]:
myDB.add_model_record(task_type = "detection", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkPP', metrics = {'accuracy':0.91}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','dog'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'accuracy':0.80}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkPP', metrics = {'accuracy':0.93}, history_address = '')
myDB.add_model_record(task_type = "classification", categories = ['cat','mouse'], model_address = './TrainedNN/New_NetworkFirst', metrics = {'loss':0.93}, history_address = '')

## Пример запроса к базе - ищем модель, удовлетворяющую критериям: "минимальная точность - 0.92, категории, с которыми работает модель - кошки, мыши". Поиск идет на включение категорий, без точного совпадения

In [6]:
myDB.get_models_by_filter({'min_metrics':{'accuracy':0.92}, 'categories':['cat','mouse']})

,ID,model_address,task_type,aux,ID,metric_name,metric_value,model_id,history_address,aux
0,2,C:\Users\fires\repos\ann-automl\TrainedNN\New_...,classification,,4,accuracy,0.93,2,C:\Users\fires\repos\ann-automl,


## Пример запроса к базе - получаем перечень всех доступных категорий

In [7]:
myDB.get_all_categories()

,ID,supercategory,name,aux
0,1,person,person,
1,2,vehicle,bicycle,
2,3,vehicle,car,
3,4,vehicle,motorcycle,
4,5,vehicle,airplane,
...,...,...,...,...
75,86,indoor,vase,
76,87,indoor,scissors,
77,88,indoor,teddy bear,
78,89,indoor,hair drier,


## Пример запроса к базе - получаем аннотации для двух категорий - кошки и собаки

In [8]:
myDB.load_specific_categories_annotations(['cat','dog'])

Train shape: (21165, 7)  test shape: (7056, 7) validation shape (7055, 7)


(              file_name                                           coco_url  \
 0             cat.0.jpg                                                      
 1             cat.1.jpg                                                      
 2            cat.10.jpg                                                      
 3           cat.100.jpg                                                      
 4          cat.1000.jpg                                                      
 ...                 ...                                                ...   
 35271  000000315555.jpg  http://images.cocodataset.org/train2017/000000...   
 35272  000000113762.jpg  http://images.cocodataset.org/train2017/000000...   
 35273  000000244933.jpg  http://images.cocodataset.org/train2017/000000...   
 35274  000000092869.jpg  http://images.cocodataset.org/train2017/000000...   
 35275  000000157804.jpg  http://images.cocodataset.org/train2017/000000...   
 
        category_id                 bbox  \
 0    

## Пример запроса к базе - получаем список всех подгруженных датасетов

In [9]:
myDB.get_all_datasets()

,ID,description,url,version,year,contributor,date_created,aux
0,1,Dogs vs Cats,https://www.kaggle.com/c/dogs-vs-cats,1.0,2013,Asirra,2013/09/25,
1,2,COCO 2017 Dataset,http://cocodataset.org,1.0,2017,COCO Consortium,2017/09/01,


## Пример запроса к базе - подгружаем аннотации из определенных датасетов (по ID)

In [10]:
myDB.load_specific_datasets_annotations([1]) #ID 1 как видно выше - это база Kaggle

,file_name,category_id,bbox,segmentation
0,cat.0.jpg,17,,
1,cat.1.jpg,17,,
2,cat.10.jpg,17,,
3,cat.100.jpg,17,,
4,cat.1000.jpg,17,,
...,...,...,...,...
24995,dog.9995.jpg,18,,
24996,dog.9996.jpg,18,,
24997,dog.9997.jpg,18,,
24998,dog.9998.jpg,18,,


# Пример запроса к базе - получить ID категорий по названию и наоборот

In [5]:
cat_IDs = myDB.get_cat_IDs_by_names(["cat","dog","fish","house","bird","person"])
print("cat_IDs:",cat_IDs)
cat_names = myDB.get_cat_names_by_IDs(cat_IDs)
print("cat_names:",cat_names)

cat_IDs: [17, 18, -1, -1, 16, 1]
cat_names: ['cat', 'dog', '', '', 'bird', 'person']
